In [1]:
import os
import sys
#from vit_pytorch import ViT
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tdata
import numpy as np
from tqdm import tqdm
import cv2


In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from cancer_classifier.config import MODELS_DIR, RAW_DATA_DIR, PROCESSED_DATA_DIR, INTERIM_DATA_DIR, CLASSES
from cancer_classifier.processing.image_utils import adjust_image_contrast, resize_image_tensor, normalize_image_tensor, augment_image_tensor, process_dataset, save_processed_images, crop_image

%load_ext autoreload
%autoreload 2

2025-05-16 19:24:37.638 | INFO     | cancer_classifier.config:<module>:11 - PROJ_ROOT path is: /home/ayoubvip/deep-learning-cancer-classifier


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## Data Preprocessing

In [4]:
img_size = (256, 256)
clip_limit = float(2.0)
tile_size = (1,1)
print(f"clip_limit = {clip_limit} ({type(clip_limit)})")

clip_limit = 2.0 (<class 'float'>)


### data crop and contract equilazing

In [5]:
for i, cls in enumerate(CLASSES):
    cls_dir = os.path.join(RAW_DATA_DIR, cls)
    for img_name in os.listdir(cls_dir):
        img_path = os.path.join(cls_dir, img_name)
        img = crop_image(img_path, tile_size, clip_limit)
        # img = adjust_image_contrast(img)

        # save in processed directory
        cv2.imwrite(os.path.join(INTERIM_DATA_DIR, cls, img_name), img)
        # save_processed_images(img_tensor, os.path.join(PROCESSED_DATA_DIR, cls), img_name)

### data augmentation

In [6]:
transformers = torchvision.transforms.Compose([
    torchvision.transforms.Grayscale(num_output_channels=1),
    torchvision.transforms.Resize(size=img_size),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.RandomRotation(10),
    torchvision.transforms.RandomResizedCrop(size=img_size, scale=(0.8, 1.0)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5], std=[0.5])
])

dataset = torchvision.datasets.ImageFolder(root=RAW_DATA_DIR, transform=transformers, target_transform=None)

### data spliting 

In [7]:
batch_nbr = 127
train_ratio = 0.80
test_ratio = 0.10
val_ratio = 0.10

In [8]:
rand_gen = torch.Generator().manual_seed(142)
train_dataset, val_dataset, test_dataset = tdata.random_split(
    dataset = dataset,
    lengths=[train_ratio, val_ratio, test_ratio],
    generator=rand_gen
)

train_loader = tdata.DataLoader(
    dataset=train_dataset,
    batch_size=batch_nbr,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)
val_loader = tdata.DataLoader(
    dataset=val_dataset,
    batch_size=batch_nbr,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
test_loader = tdata.DataLoader(
    dataset=test_dataset,
    batch_size=batch_nbr,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
def get_class_distribution(dataset):
    class_distribution = {}
    for _, label in dataset:
        if label.__str__() not in class_distribution:
            class_distribution[label.__str__()] = 0
        class_distribution[label.__str__()] += 1
    return class_distribution

print("Train dataset class distribution:", get_class_distribution(train_dataset))
print("Validation dataset class distribution:", get_class_distribution(val_dataset))
print("Test dataset class distribution:", get_class_distribution(test_dataset))

Train dataset class distribution: {'2': 1640, '1': 1601, '0': 1604}
Validation dataset class distribution: {'1': 211, '2': 209, '0': 186}
Test dataset class distribution: {'0': 214, '2': 199, '1': 192}


# Model Architecture: Vision Transformer (Dosovitskiy et al., 2021)

In [9]:
def conv_block(in_channels, out_channels, kernel_size=3,
                stride=1, padding=1, pool_kernel_size=2,
                pool_stride=2, dropout_prob=0.2):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        nn.Dropout2d(p=dropout_prob),
        nn.MaxPool2d(pool_kernel_size, pool_stride)
    )

In [10]:
class CNNModel(nn.Module):
  def __init__(self, num_classes=3):
    super(CNNModel, self).__init__()

    self.conv_block1 = conv_block(1, 64)
    self.conv_block2 = conv_block(64, 128)
    self.conv_block3 = conv_block(128, 256)
    self.conv_block4 = conv_block(256, 512)

    dummy_input = torch.randn(1, 1, 256, 256) # Sadece bir görsel için çıkış boyutu hesaplar
    dummy_output = self.conv_block4(self.conv_block3(self.conv_block2(self.conv_block1(dummy_input))))

    flattened_size = torch.flatten(dummy_output, 1).size(1)

    print(f'the full architecture parameters size: {flattened_size}')

    self.fc_layers = nn.Sequential(
           nn.Dropout(0.3),
           nn.Linear(flattened_size, 512), # Hesaplanan boyutu kullan
           nn.ReLU(inplace=True),
           nn.Linear(512, num_classes)
        )

  def forward(self, x):
    x = self.conv_block1(x)
    x = self.conv_block2(x)
    x = self.conv_block3(x)
    x = self.conv_block4(x)

    x = torch.flatten(x, 1)

    x = self.fc_layers(x)

    return x


In [11]:
CNN_model = CNNModel(num_classes=len(CLASSES)).to(device)

the full architecture parameters size: 131072


## Training the model

### defining Loss function, Optimization method, and training parameters

In [12]:
loss_fn = nn.CrossEntropyLoss()
weights_decay = 0.0001
learning_rate = 0.0001
epochs = 5

# optimizer = torch.optim.Adam(
#     CNN_model.parameters(),
#     lr=learning_rate,
#     weight_decay=weights_decay
# ) #->70%

optimizer = torch.optim.AdamW(
    CNN_model.parameters(),
    lr=learning_rate,
    weight_decay=weights_decay
) #->77.4

# optimizer = torch.optim.SGD(
#     CNN_model.parameters(),
#     lr=learning_rate,
#     weight_decay=weights_decay
# ) #-> 77.5

In [13]:
from cancer_classifier.modeling.train import train
# import evaluate from cancer_classifier.modeling.evaluate as evaluate
def train_loop(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    print("----------------- EPOCH " + str(epoch) + "------------------")
    for batch, (X, y) in tqdm(enumerate(dataloader)):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device) 
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_nbr + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

for epoch in range(epochs):
    train_loop(
        dataloader=train_loader,
        model=CNN_model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        epoch=epoch
        )


----------------- EPOCH 0------------------


0it [00:01, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1016.00 MiB. GPU 0 has a total capacity of 10.75 GiB of which 336.69 MiB is free. Process 2045031 has 2.49 GiB memory in use. Including non-PyTorch memory, this process has 7.92 GiB memory in use. Of the allocated memory 7.73 GiB is allocated by PyTorch, and 16.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    true_labels = []
    pred_labels = []
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            true_labels.extend(y.cpu().numpy())
            pred_labels.extend(pred.argmax(1).cpu().numpy())

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    return true_labels, pred_labels, correct, test_loss


true_labels, pred_labels, correct, test_loss =  test_loop(
    dataloader=test_loader,
    model=CNN_model,
    loss_fn=loss_fn
)
# train(
#     model=CNN_model,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     test_loader=test_loader,
#     loss_fn=loss_fn,
#     optimizer=optimizer,
#     epochs=epochs,
#     device=device
# )

## Evaluating the model

### Covariance Matrix

In [ ]:
from cancer_classifier.plots import visualize_sample_images,  plot_confusion_matrix
import time

visualize_sample_images(
    dataset=dataset,
)
plot_confusion_matrix(
    true_classes=true_labels,
    predicted_classes=pred_labels,
    model_name =  time.strftime("%Y-%m-%d_%H-%M-%S") + "_CNN_model_" + f"{(100*correct):>0.1f}%"
)

### serializing the model

In [ ]:
import time
model_name = f"{(100*correct):>0.1f}%--" + time.strftime("%Y-%m-%d_%H-%M-%S") + "_CNN_model.pth"
torch.save(CNN_model.state_dict(), MODELS_DIR / model_name)
print("Saved PyTorch Model State to " +  model_name)

## References

- https://d2l.ai/chapter_attention-mechanisms-and-transformers/vision-transformer.html
- https://github.com/lucidrains/vit-pytorch
- https://docs.pytorch.org/tutorials/beginner/basics/transforms_tutorial.html
